# Imputacion
Notebook con ejemplos de uso de la funcion imputacion

## importaciones
Nota: estas importaciones son necesarias para este notebook, si quiere ver las importaciones necesarias para la función específica vaya al archivo original [imputacion](../features/imputacion.py)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

## dataframes y validaciones
Estos son dataframes de testeo obtenido de internet. 
Las validaciones son una pieza importante de la parte técnica de la librería. Para más detalle vea las [validaciones](../validaciones/validaciones.py)

In [14]:
df_null = spark.read.csv('csv/cars_null.csv' , header= True  ,inferSchema=True)

def is_dataframe(dataframe):
    try:
        type_df = type(dataframe)
        if not isinstance(dataframe , DataFrame):
            raise TypeError(f"Expected a DataFrame, got {type_df}")
    except Exception as e: 
        print("An error occurred: ", e)
def sel_num_cols(dataframe): 
    lista_columnas_numericas =  []
    tipos_numericos = [LongType().simpleString(), DoubleType().simpleString(), 
        IntegerType().simpleString() , ShortType().simpleString() ,
        FloatType().simpleString() , DecimalType().simpleString()]
    for columnas, dtype in dataframe.dtypes:
        if dtype in tipos_numericos: lista_columnas_numericas.append(columnas)
    
    return lista_columnas_numericas
def df_has_numtype(dataframe):
    try:
        is_dataframe(dataframe)
        tipos_numericos = [LongType().simpleString(), DoubleType().simpleString(), 
        IntegerType().simpleString() , ShortType().simpleString() ,
        FloatType().simpleString() , DecimalType().simpleString()]
        for _ , dtype in dataframe.dtypes: 
            if dtype in tipos_numericos:  return True 
        return False
    except Exception as e:
        print("An error occurred: ", e)
def df_has_null(dataframe): 
    try:    
        is_dataframe(dataframe)
        for col in dataframe.columns:
            if dataframe.filter(isnull(col)).count() > 0 : return (True , col)
        return (False , None)
    except Exception as e: 
        print("An error occurred: ", e)

## Vista previa del dataframes con datos nulos

In [15]:
df_null.show(5)

+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|       null|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|  3|Volkswagen|   Golf|      24990|GTI 2.0 TSI 169kW...|2018|        44495|Gasolina|    Manual|Viladecans|
|  4|      Opel|  Corsa|      10460|1.4 Expression 90 CV|2016|        69800|      \n|    Manual|Sabadell 1|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
only showing top 5 rows



## inferSchema e imputacion
Utilizaremos la funcion [inferSchema](../features/inferSchema.py) para obtener el tipo de dato correcto y luego haremos la [imputacion](../features/imputacion.py) de los datos nulos

In [7]:
def inferSchema(dataframe):
    """
    Esta función recibe un dataframe de PySpark, una columna a ser imputada y una columna auxiliar. 
    La función devuelve el dataframe con la columna de imputación llenada con el promedio de los valores 
    de la columna de imputación agrupada por la columna auxiliar. Si el valor de la columna de imputación
    no es nulo, se mantiene el mismo valor.
    Argumentos:
    dataframe (DataFrame): Dataframe de PySpark que se desea imputar
    columna_imputacion (string): Nombre de la columna que se desea imputar
    columna_auxiliar (string): Nombre de la columna auxiliar para calcular el promedio
    Retorno:
    DataFrame: Dataframe de PySpark con la columna de imputación llenada con el promedio de los valores 
               de la columna de imputación agrupada por la columna auxiliar.
    """
    is_dataframe(dataframe) 
    try:
        dataframe = dataframe.select([col(c).cast("string") for c in dataframe.columns])
        for columna in dataframe.columns: 
            first_value = dataframe.select(col(columna)).first()[0]
            if contains_letter(first_value):
                dataframe = dataframe.withColumn(columna , col(columna).cast(StringType())) 
                print(f'{columna} | string -> string')
            elif columna == "Date":
                contador = first_value.count(".")
                if '-' in first_value:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(DateType()))  
                    print(f'{columna} | string -> date')
                
                elif contador == 2: 
                    dataframe = dataframe.withColumn(columna, regexp_replace(dataframe[columna], "[^a-zA-Z0-9]+", ""))
                    dataframe = dataframe.withColumn(columna , col(columna).cast((IntegerType()))) 
                    print(f'{columna} | string -> integer')
            elif '.' in first_value:
                contador = first_value.count(".")
                if contador == 2: 
                    dataframe = dataframe.withColumn(columna, regexp_replace(dataframe[columna], "[^a-zA-Z0-9]+", ""))
                    dataframe = dataframe.withColumn(columna , col(columna).cast((IntegerType()))) 
                    print(f'{columna} | string -> date')
                elif contador == 1:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(DoubleType()))
                    print(f'{columna} | string -> double') 
                else:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(StringType()))   
                    print(f'{columna} | string -> string')
                    

            elif "-"  in first_value:
                contador = first_value.count("-")
                if contador == 2:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(DateType()))  
                    print(f'{columna} | string -> date')
                else: 
                    dataframe = dataframe.withColumn(columna , col(columna).cast(StringType()))   
                    print(f'{columna} | string -> string')
            elif "/" in first_value:
                contador = first_value.count("/")
                if contador == 2:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(DateType()))
                    print(f'{columna} | string -> date')
                else: 
                    dataframe = dataframe.withColumn(columna , col(columna).cast(StringType()))   
                    print(f'{columna} | string -> string')

            elif "True" in first_value:
                dataframe = dataframe.withColumn(columna , col(columna).cast(BooleanType()))
                print(f'{columna} | string -> bool')
            elif "False" in first_value:
                dataframe = dataframe.withColumn(columna , col(columna).cast(BooleanType()))
                print(f'{columna} | string -> bool')
            elif first_value.isnumeric():
                dataframe = dataframe.withColumn(columna , col(columna).cast(LongType()))
                try:
                    dataframe = dataframe.withColumn(columna , col(columna).cast((IntegerType())))
                    print(f'{columna} | string -> integer')
                except:
                    dataframe = dataframe.withColumn(columna , col(columna).cast(LongType()))
                    print(f'{columna} | string -> long')
                    pass
            else:
                dataframe = dataframe.withColumn(columna , col(columna).cast(StringType()))   
                print(f'{columna} | string -> string')
        return dataframe
    except Exception as e :
        print('Ha ocurrido un error al momento de inferir el schema: ' , e )


def contains_letter(string):
    return bool(re.search("[a-zA-Z]", string))

In [8]:
def imputacion(dataframe , columna , opcion , auxiliar = ''):
    """
    La función 'imputacion' es una función que permite reemplazar los valores nulos en un dataframe dado por un valor determinado. 
    
    Argumentos:
    dataframe (pyspark.sql.dataframe.DataFrame): El dataframe del que se quieren reemplazar los valores nulos.
    columna (str): La columna que se desea imputar.
    opcion (str): La opción de imputación que se desea aplicar. Puede ser "media", "mediana", "moda", "similitud", "cero", o "desconocido".
    auxiliar (str, optional): El nombre de una columna auxiliar que se utilizará en caso de que la opción elegida sea "similitud". 
                             Por defecto, no se proporciona.
    Retorno:
    pyspark.sql.dataframe.DataFrame: Un nuevo dataframe con los valores nulos en la columna dada reemplazados por el valor especificado.
    
    Ejemplos:
    >>> dataframe = spark.createDataFrame([(1, None), (2, 3.0), (3, 4.0)], ["id", "val"])
    >>> imputacion(dataframe, "val", "media")
    >>> # Returns:
    >>> # +---+-----+
    >>> # | id|  val|
    >>> # +---+-----+
    >>> # |  1| 3.25|
    >>> # |  2|  3.0|
    >>> # |  3|  4.0|
    >>> # +---+-----+
    
    """
    is_dataframe(dataframe)
    try:
        
        if dataframe.filter(isnull(columna)).count() > 0 : 
            tipos_numericos = [LongType().simpleString(), DoubleType().simpleString(), 
                IntegerType().simpleString() , ShortType().simpleString() ,
                FloatType().simpleString() , DecimalType().simpleString()]
            data_type = dataframe.schema[columna].dataType
            data_type = data_type.simpleString()
            
            if data_type in tipos_numericos:
                if opcion == 'mediana':
                    print('opcion de mediana')
                    mean_val = dataframe.select(mean(columna)).first()[0]
                    dataframe = dataframe.fillna(mean_val, [columna])
                    return dataframe
                elif opcion == 'media':
                    print('opcion de media')
                    median_val = dataframe.select(percentile_approx(columna, 0.5).alias("median")).first()[0]
                    dataframe = dataframe.fillna(median_val, [columna])
                    return dataframe
                elif opcion == 'moda':
                    print('opcion de moda')
                    mode_val = dataframe.groupBy(columna).count().sort('count', ascending=False).first()[0]
                    dataframe = dataframe.fillna(mode_val, [columna])
                    return dataframe
                elif opcion == 'similitud':
                    df_fill = similitud(dataframe , columna , auxiliar)
                    return df_fill
                elif opcion == 'cero':
                    dataframe = dataframe.fillna(0, [columna])
                    return dataframe
                            
            elif data_type == StringType().simpleString():
                lista_nulls = ['null' , '-' , '' , ' ', 'n' , '\\n' ]        
                if opcion == 'moda': 
                    print('opcion de moda')
                    mode_val = dataframe.filter(dataframe[columna].isNotNull()).groupBy(columna).count().sort('count', ascending=False).first()[0]
                    for i, row in enumerate(dataframe.toLocalIterator()):
                        if row[columna] in lista_nulls:
                            dataframe = dataframe.withColumn(columna, when(col(columna).isin(lista_nulls), mode_val).otherwise(col(columna)))  
                    return dataframe 
                elif opcion == 'cero': 
                    print('opcion de cero')
                    for i, row in enumerate(dataframe.toLocalIterator()):
                        if row[columna] in lista_nulls:
                            dataframe = dataframe.withColumn(columna, when(col(columna).isin(lista_nulls), '0').otherwise(col(columna)))  
                    return dataframe
                elif opcion == 'desconocido':
                    print('opcion de desconocido')
                    for i, row in enumerate(dataframe.toLocalIterator()):
                        if row[columna] in lista_nulls:
                            dataframe = dataframe.withColumn(columna, when(col(columna).isin(lista_nulls), 'desconocido').otherwise(col(columna)))  
                    return dataframe
        else: 
            print('No existen valores nulos en la columna indicada')
            return dataframe
    except Exception as e : 
        print('Error: ' , e)

def similitud(dataframe, columna_imputacion, columna_auxiliar):
    """
    Esta función recibe un dataframe de PySpark, una columna a ser imputada y una columna auxiliar. 
    La función devuelve el dataframe con la columna de imputación llenada con el promedio de los valores 
    de la columna de imputación agrupados por la columna auxiliar. Si el valor de la columna de imputación
    no es nulo, se mantiene el mismo valor.
    Parameters:
    dataframe (DataFrame): Dataframe de PySpark que se desea imputar
    columna_imputacion (string): Nombre de la columna que se desea imputar
    columna_auxiliar (string): Nombre de la columna auxiliar para calcular el promedio
    Returns:
    DataFrame: Dataframe de PySpark con la columna de imputación llenada con el promedio de los valores 
               de la columna de imputación agrupados por la columna auxiliar.
    """
    try:
        avg_vals = dataframe.filter(dataframe[columna_imputacion].isNotNull()).groupBy(columna_auxiliar).agg(avg(columna_imputacion))
        df_fill = dataframe.join(avg_vals, columna_auxiliar, 'left') \
                           .withColumn(columna_imputacion, when(dataframe[columna_imputacion].isNull(), col('avg(' + columna_imputacion + ')')).otherwise(dataframe[columna_imputacion])) \
                           .drop('avg(' + columna_imputacion + ')')
        return df_fill
    except Exception as e:
        print("Error: ", e)

### Ejemplo columna String imputacion moda
Este ejemplo rellenará los datos nulos de una columna tipo String con el valor que más se repite

In [17]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'fuel' , 'moda')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
opcion de moda
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|       null|1.6 CRDi 110cv Tecno|2014|       104868|  Diésel|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|  Diésel|Automatica|Viladecans|
|  3|Volkswagen|   Golf|      24990|GTI 2.0 TSI 169kW...|2018|     

### Ejemplo columna String imputacion cero
Este ejemplo rellenará con '0' los datos que sean nulos en la columna fuel


In [18]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'fuel' , 'cero')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
opcion de cero
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|       null|1.6 CRDi 110cv Tecno|2014|       104868|       0|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       0|Automatica|Viladecans|
|  3|Volkswagen|   Golf|      24990|GTI 2.0 TSI 169kW...|2018|     

### Ejemplo columna String imputacion deconocido

Este ejemplo rellenará con la palabra 'desconocido' los datos que sean nulo en la columna fuel 

In [24]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'fuel' , 'desconocido')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
opcion de desconocido
+---+----------+-------+-----------+--------------------+----+-------------+-----------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|       fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+-----------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|   Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|       null|1.6 CRDi 110cv Tecno|2014|       104868|desconocido|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|desconocido|Automatica|Viladecans|
|  3|Volkswagen|   Golf|      24990|GTI 2.

### Ejemplo columna numerica imputacion mediana
Este ejemplo rellenará los datos nulos de las columnas numéricas con la mediana de los datos de la columna 'price (eur)'

In [26]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'price (eur)' , 'mediana')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
estoy dentro de el if de datos numericos
opcion de mediana
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|      15995|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|  3|Volkswagen|   Golf

### Ejemplo columna numerica imputacion moda
Este ejemplo rellenará los datos nulos de las columnas numéricas con la moda de los datos de la columna 'price (eur)'

In [27]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'price (eur)' , 'moda')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
estoy dentro de el if de datos numericos
opcion de moda
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|      14990|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|  3|Volkswagen|   Golf|  

### Ejemplo columna numerica imputacion media
Este ejemplo rellenará los datos nulos de las columnas numéricas con la media de los datos de la columna 'price (eur)'

In [28]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'price (eur)' , 'media')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
estoy dentro de el if de datos numericos
opcion de media
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|      14990|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|  3|Volkswagen|   Golf| 

### Ejemplo columna numerica imputacion similitud
Este ejemplo rellenará los datos nulos de las columnas numéricas con la similitud de los datos de la columna 'price (eur)' tomando los valores de referencia del valor específico en 'model'



In [30]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'price (eur)' , 'similitud' , 'model')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
estoy dentro de el if de datos numericos
+-------+---+----------+-----------+--------------------+----+-------------+--------+----------+----------+
|  model|_c0|     brand|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+-------+---+----------+-----------+--------------------+----+-------------+--------+----------+----------+
|  Ibiza|  0|      SEAT|     8990.0|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|    i30|  1|   Hyundai|    15684.0|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|Serie 5|  2|       BMW|    13490.0|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|   Golf|  3|Volkswagen|    24990.0|GTI 2

### Ejemplo columna numerica imputacion moda
Este ejemplo rellenará los datos nulos de las columnas numéricas con la moda de los datos de la columna 'price (eur)'



In [31]:
df_infer_cars = inferSchema(df_null)
df_limpio_cars = imputacion(df_infer_cars , 'model' , 'moda')
df_limpio_cars.show(5)

_c0 | string -> integer
brand | string -> string
model | string -> string
price (eur) | string -> integer
engine | string -> string
year | string -> integer
mileage (kms) | string -> integer
fuel | string -> string
gearbox | string -> string
location | string -> string
No existen valores nulos en la columna indicada
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|_c0|     brand|  model|price (eur)|              engine|year|mileage (kms)|    fuel|   gearbox|  location|
+---+----------+-------+-----------+--------------------+----+-------------+--------+----------+----------+
|  0|      SEAT|  Ibiza|       8990|SC 1.2 TSI 90cv S...|2016|        67000|Gasolina|    Manual|Granollers|
|  1|   Hyundai|    i30|       null|1.6 CRDi 110cv Tecno|2014|       104868|    null|    Manual|Viladecans|
|  2|       BMW|Serie 5|      13490|        530d Touring|2011|       137566|       -|Automatica|Viladecans|
|  3|Volkswagen|   Golf|      2499